# credit card transaction fraud

# loading data

In [1]:
import pandas as pd
import os

training_data_path = "fraudTrain.csv"
test_data_path = "fraudTest.csv"

training_data_raw = pd.read_csv(training_data_path)
test_data_raw = pd.read_csv(test_data_path)

# cleaning data

In [2]:
#data cleaning here
training_data_clean = {"cc_num": [], "merchant": [], "category": [], "amt": [], "first": [], "last": [], "gender": [], "lat": [], "long": [], "city_pop": [], "job": [], "unix_time": [], "merch_lat": [], "merch_long": [], "is_fraud": []}
for idx in range(len(training_data_raw["cc_num"])):
    for key in training_data_clean.keys():
        training_data_clean[key].append(training_data_raw[key][idx])
training_data_clean = pd.DataFrame.from_dict(training_data_clean)
training_data_clean.to_csv("credit_card_fraud_clean_train.csv", index=False)

test_data_clean = {"cc_num": [], "merchant": [], "category": [], "amt": [], "first": [], "last": [], "gender": [], "lat": [], "long": [], "city_pop": [], "job": [], "unix_time": [], "merch_lat": [], "merch_long": [], "is_fraud": []}
for idx in range(len(test_data_raw["cc_num"])):
    for key in test_data_clean.keys():
        test_data_clean[key].append(test_data_raw[key][idx])
test_data_clean = pd.DataFrame.from_dict(test_data_clean)
test_data_clean.to_csv("credit_card_fraud_clean_test.csv", index=False)

# Preprocess data for neural network

In [3]:
#Handle subselection of columns for the neural network
#columns to keep: cc_num, merchant, category, amt, lat, long, unix_time, merch_lat, merch_long, is_fraud

#find unique merchants
unique_merchants = set(list(training_data_clean["merchant"]))
for merchant in list(test_data_clean["merchant"]):
    unique_merchants.add(merchant)
unique_merchants = list(unique_merchants)
print(len(unique_merchants))

#create tokenization scheme for merchants
merchant_tokenized_dict = {}
for idx in range(len(unique_merchants)):
    merchant_tokenized_dict[unique_merchants[idx]] = idx

#find unique categories
unique_categories = set(list(training_data_clean["category"]))
for category in list(test_data_clean["category"]):
    unique_categories.add(category)
unique_categories = list(unique_categories)
print(len(unique_categories))

#create tokenization scheme for categories
categories_tokenized_dict = {}
for idx in range(len(unique_categories)):
    categories_tokenized_dict[unique_categories[idx]] = idx

#columns to keep: merchant, category, amt, lat, long, unix_time, merch_lat, merch_long, is_fraud
#create non-categorical data list
non_categorical = []
for idx in range(len(training_data_clean["cc_num"])):
    element = []
    #element.append(training_data_clean["cc_num"][idx])
    element.append(training_data_clean["amt"][idx])
    element.append(training_data_clean["lat"][idx])
    element.append(training_data_clean["long"][idx])
    element.append(training_data_clean["unix_time"][idx])
    element.append(training_data_clean["merch_lat"][idx])
    element.append(training_data_clean["merch_long"][idx])
    non_categorical.append(element)
print("Non-categorical data length:", len(non_categorical))

from keras.utils import to_categorical
#format merchant column as one-hot list
num_unique_merchants = len(unique_merchants)
merchant_categorical = []
for idx in range(len(training_data_clean["merchant"])):
    element_raw = training_data_clean["merchant"][idx]
    element_int = merchant_tokenized_dict[element_raw]
    merchant_categorical.append(to_categorical(element_int, num_classes=num_unique_merchants))
print("Merchant data length:", len(merchant_categorical))

#format category data as one-hot list
num_unique_categories = len(unique_categories)
category_one_hot = []
for idx in range(len(training_data_clean["category"])):
    element_raw = training_data_clean["category"][idx]
    element_int = categories_tokenized_dict[element_raw]
    category_one_hot.append(to_categorical(element_int, num_classes=num_unique_categories))
print("Category length:", len(category_one_hot))

#create is_fraud list as y column
fraud_column = []
num_fraud = 0
for idx in range(len(training_data_clean["is_fraud"])):
    fraud_val = training_data_clean["is_fraud"][idx]
    num_fraud += fraud_val
    fraud_column.append([fraud_val])
print("Fraud length:", len(fraud_column))
print("Number of fraudulent transactions:", num_fraud)

693
14
Non-categorical data length: 1296675


Using TensorFlow backend.
/Users/Jerome/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/Jerome/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/Jerome/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/Jerome/opt/anaconda3/lib/python3.7

Merchant data length: 1296675
Category length: 1296675
Fraud length: 1296675
Number of fraudulent transactions: 7506


# subsample data

In [4]:
import random
##find all fraud in training data
fraud_data_merchant_sample = []
fraud_data_category_sample = []
fraud_data_non_categorical_sample = []
fraud_data_label_sample = []
for idx in range(len(training_data_clean["is_fraud"])):
    #if the row at idx represents fraud
    if(training_data_clean["is_fraud"][idx] == 1):
        #add data to fraud data samples according to data type
        fraud_data_non_categorical_sample.append(non_categorical[idx])
        fraud_data_category_sample.append(category_one_hot[idx])
        fraud_data_merchant_sample.append(merchant_categorical[idx])
        fraud_data_label_sample.append([1])
##add subsample of non-fraud data with equal number of rows to fraud data in training data
max_index = len(training_data_clean["is_fraud"]) - 1
num_safe_transactions_included = 0
while num_safe_transactions_included < num_fraud:
    #select random row index in the formatted fraud data above
    random_index = random.randint(0, max_index)
    #if this data does not represent a fraudulent transaction
    if(training_data_clean["is_fraud"][random_index] == 0):
        #add the data to fraud_data samples accordingly
        fraud_data_non_categorical_sample.append(non_categorical[random_index])
        fraud_data_category_sample.append(category_one_hot[random_index])
        fraud_data_merchant_sample.append(merchant_categorical[random_index])
        fraud_data_label_sample.append([0])
    num_safe_transactions_included += 1

import numpy as np

fraud_data_non_categorical_sample = np.array(fraud_data_non_categorical_sample)
print("Non-categorical shape:", fraud_data_non_categorical_sample.shape)
fraud_data_merchant_sample = np.array(fraud_data_merchant_sample)
print("Merchant shape:", fraud_data_merchant_sample.shape)
fraud_data_category_sample = np.array(fraud_data_category_sample)
print("Category shape:", fraud_data_category_sample.shape)
fraud_data_label_sample = np.array(fraud_data_label_sample)
print("Label shape:", fraud_data_label_sample.shape)

Non-categorical shape: (14970, 6)
Merchant shape: (14970, 693)
Category shape: (14970, 14)
Label shape: (14970, 1)


# Keras environment setup

In [9]:
#load ML libraries
import hyperopt

from hyperopt import fmin, tpe

from keras.models import Model, load_model
from keras.layers import Dense, Dropout, Input, concatenate

from keras import activations
from keras import callbacks
from keras import regularizers
from keras.utils import plot_model

from sklearn.model_selection import KFold

#define method for generating model
def define_model(num_nodes, dropout, kernel_reg, bias_reg):
    #define input nodes
    input_non_categorical = Input(shape=(6,))
    input_merchant_category = Input(shape=(693,))
    input_purchase_category = Input(shape=(14,))
    #define processing
    concat_input = concatenate(inputs=[input_non_categorical, input_merchant_category, input_purchase_category])
    dense_layer_1 = Dense(num_nodes, activation=activations.relu, kernel_regularizer=regularizers.l2(kernel_reg), bias_regularizer=regularizers.l2(bias_reg))(concat_input)
    dropout_layer_1 = Dropout(dropout)(dense_layer_1)
    #dense_layer_2 = Dense(num_nodes // 2, activation=activations.relu, kernel_regularizer=regularizers.l2(kernel_reg), bias_regularizer=regularizers.l2(bias_reg))(dropout_layer_1)
    #concat_tail = concatenate(inputs=[dropout_layer_1, input_non_categorical, input_purchase_category])
    #dropout_layer_2 = Dropout(dropout / 2)(concat_tail)
    #dense_layer_3 = Dense(num_nodes // 2, activation=activations.relu, kernel_regularizer=regularizers.l2(kernel_reg), bias_regularizer=regularizers.l2(bias_reg))(dropout_layer_2)
    #define output
    #output = Dense(1, activation=activations.sigmoid)(concat_tail)
    output = Dense(1, activation=activations.sigmoid)(dropout_layer_1)
    #compile and return
    network = Model(inputs=[input_non_categorical, input_merchant_category, input_purchase_category], outputs=output)
    #network = Model(inputs=[input_non_categorical, input_purchase_category], outputs=output)
    network.compile(loss="binary_crossentropy")
    return network

# hyperparameter Tuning

In [10]:
#do hyperparameter tuning here
import math
import numpy as np

best_loss = math.inf
best_params = ()

def average(data_list):
    avg = 0
    for idx in range(len(data_list)):
        if not np.isnan(data_list[idx]):
            avg += data_list[idx]
        else:
            avg += 10
    avg /= len(data_list)
    return avg

def get_average_cross_validation_accuracy(args):
    global best_loss, best_params
    max_node_count, dropout, kernel_regularization, bias_regularization = args
    data_split = KFold(n_splits=5, shuffle=True)
    loss_vals = []
    callbacks_list = [callbacks.EarlyStopping(monitor='val_loss', patience=5,), callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=1,)]
    for train_index, test_index in data_split.split(fraud_data_non_categorical_sample, fraud_data_label_sample):
        #convert data into train and test data
        non_categorical_train, non_categorical_test = fraud_data_non_categorical_sample[train_index], fraud_data_non_categorical_sample[test_index]
        merchant_cat_train, merchant_cat_test = fraud_data_merchant_sample[train_index], fraud_data_merchant_sample[test_index]
        purchase_cat_train, purchase_cat_test = fraud_data_category_sample[train_index], fraud_data_category_sample[test_index]
        label_train, label_test = fraud_data_label_sample[train_index], fraud_data_label_sample[test_index]
        network = define_model(max_node_count, dropout, kernel_regularization, bias_regularization)
        network.fit(x=[non_categorical_train, merchant_cat_train, purchase_cat_train], y=label_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks_list, verbose=0)
        evaluation = network.evaluate(x=[non_categorical_test, merchant_cat_test, purchase_cat_test], y=label_test, verbose=0)
        #network.fit(x=[non_categorical_train, purchase_cat_train], y=label_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks_list, verbose=0)
        #evaluation = network.evaluate(x=[non_categorical_test, purchase_cat_test], y=label_test, verbose=0)
        loss_vals.append(evaluation)
        del network
        del non_categorical_train, non_categorical_test
        del merchant_cat_train, merchant_cat_test
        del label_train, label_test
    average_loss = average(loss_vals)
    if average_loss < best_loss:
        best_params = args
        best_loss = average_loss
    return average_loss

def optimize_hyperparameters():
    max_node_counts = [32, 64, 128, 256, 512]
    dropout_values = [0.2, 0.3, 0.4, 0.5, 0.6]
    kernel_reg_list = [0.01, 0.025, 0.05]
    bias_reg_list = [0.01, 0.025, 0.05]
    case_list = []
    for max_node_count in max_node_counts:
        for dropout_value in dropout_values:
            for kernel_reg in kernel_reg_list:
                for bias_reg in bias_reg_list:
                    case_list.append((max_node_count, dropout_value, kernel_reg, bias_reg))
    feature_space = hyperopt.hp.choice('a', case_list)
    fmin(get_average_cross_validation_accuracy, feature_space, algo=tpe.suggest, max_evals=128)

optimize_hyperparameters()
print("Best parameters:", best_params)
print("Best loss:", best_loss)

100%|██████████| 128/128 [23:11<00:00, 10.87s/trial, best loss: 0.685332453250885]
Best parameters: (32, 0.2, 0.025, 0.025)
Best loss: 0.685332453250885


# Train Neural Network

In [31]:
#Train neural network here
import os
#os.mkdir("cc_fraud_detector")

callbacks_list = [callbacks.EarlyStopping(monitor='val_loss', patience=5,)]
try:
    max_node_count, dropout_value, kernel_reg, bias_reg = best_params
except:
    max_node_count, dropout_value, kernel_reg, bias_reg = 32, 0.2, 0.025, 0.025

network = define_model(max_node_count, dropout_value, kernel_reg, bias_reg)
history = network.fit(x=[fraud_data_non_categorical_sample, fraud_data_merchant_sample, fraud_data_category_sample], y=fraud_data_label_sample, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks_list)
#history = network.fit(x=[fraud_data_non_categorical_sample, fraud_data_category_sample], y=fraud_data_label_sample, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks_list)
plot_model(network, to_file='cc_fraud_detector.png', show_shapes=True)

Train on 11976 samples, validate on 2994 samples
Epoch 1/20
11976/11976 [==============================] - 1s 107us/step - loss: 6999204.8599 - val_loss: 2048748.4890
Epoch 2/20
11976/11976 [==============================] - 0s 35us/step - loss: 1249072.1383 - val_loss: 0.0266
Epoch 3/20
11976/11976 [==============================] - 0s 36us/step - loss: 29302.3100 - val_loss: 0.0229
Epoch 4/20
11976/11976 [==============================] - 0s 36us/step - loss: 2000.6421 - val_loss: 1876.2998
Epoch 5/20
11976/11976 [==============================] - 0s 35us/step - loss: 1398.7321 - val_loss: 3353.5233
Epoch 6/20
11976/11976 [==============================] - 0s 36us/step - loss: 820.8241 - val_loss: 0.0207
Epoch 7/20
11976/11976 [==============================] - 0s 36us/step - loss: 786.7415 - val_loss: 0.0204
Epoch 8/20
11976/11976 [==============================] - 0s 36us/step - loss: 954.4176 - val_loss: 4143.0151
Epoch 9/20
11976/11976 [==============================] - 0s 35us/s

# False positive and negative rates

In [23]:
#make predictions based on ALL data rows (1296675 rows)
prediction = network.predict([non_categorical, merchant_categorical, category_one_hot], batch_size=128, verbose=1, use_multiprocessing=True)
prediction = prediction.reshape((1296675))
prediction = prediction.tolist()
truth_values = np.array(training_data_clean["is_fraud"]).reshape((1296675))
truth_values = truth_values.tolist()
#create counts for false pos, false negative, true pos, and true negative
false_pos_count = 0
false_negative_count = 0
true_pos_count = 0
true_negative_count = 0
#for each prediction
for idx in range(1296675):
    #compare each prediction to the true value
    #increment appropriate count
    if(abs(prediction[idx] - truth_values[idx]) < 0.5):
        if(int(truth_values[idx]) == 0):
            true_negative_count += 1
        else:
            true_pos_count += 1
    else:
        if(int(truth_values[idx]) == 0):
            false_pos_count += 1
        else:
            false_negative_count += 1
#output
print("True positives:", true_pos_count)
print("True negatives:", true_negative_count)
print("False positives:", false_pos_count)
print("False negatives:", false_negative_count)

1296675/1296675 [==============================] - 8s 6us/step
True positives: 7506
True negatives: 0
False positives: 1289169
False negatives: 0


# Evaluate on test set

In [29]:
#evaluate
print(network.evaluate([non_categorical, merchant_categorical, category_one_hot], np.array(training_data_clean["is_fraud"]), verbose=0))
print(history.history)

1.0415509112926966
{'val_loss': [986944.9451402805, 20466.413291426605, 0.017238952219486237, 15944.724000735847, 0.01613517850637436, 13362.053722027387, 294.20599477356404, 0.015318620949983597, 2466.092094997286, 3440.411817123831, 0.014605141244828701, 0.014344729483127594, 185.6782540096469, 27.894352156716185, 1.435651861673685, 1.3124525739737327, 1.0449346056283915], 'loss': [7020387.8559619235, 304408.5956783358, 11525.634831348896, 5185.692391129916, 2840.280045523991, 1876.2393422913535, 1310.6587496281625, 1702.8371943268046, 1101.9295978839189, 932.8935347093927, 410.93460947366736, 189.7249097906914, 71.31185670494635, 42.09237258818123, 1.5221927117807672, 0.7182963884665159, 0.746050506571411]}
